### Environment & Visualization

Initializes the 2D environment and prints agent sizes as growing 🟩 bars with movement history after each generation.

In [2]:
# environment.py

def return_environment(x_max, y_max):
    env = [x_max, y_max]
    return env

def print_grid(list_of_agents, env):
    x_max, y_max = env

    # # Create an empty grid
    # grid = [["." for _ in range(x_max)] for _ in range(y_max)]

    # # Place each agent in the grid
    # for agent in list_of_agents:
    #     x, y = agent["position"]
    #     agent_id = agent["id"][-1]  # gets '1' or '2' from 'agent_1', etc.
    #     grid[y][x] = agent_id  # Note: row = y, col = x

    # # Print the grid
    # print("Grid:")
    # for row in grid:
    #     print(" ".join(row))

    # Visual size bars for each agent
    print("\nAgent Sizes:")
    for agent in list_of_agents:
        bar_length = int(agent["size"] // 2)  # one block per 2 units of size
        size_bar = "🟩" * bar_length
        print(f"{agent['id']}: {size_bar} ({agent['size']:.2f}) ({agent['movements']})")


### Gene Logic: Size & Movement

Defines how genes influence development:
- `size_gene_rule` adjusts size based on even/odd logic.
- `movement_generation_gene` biases direction choice using past movement frequency (parity-based weighting).

In [3]:
# gene_generation.py

from collections import Counter
import random

def size_gene_rule(agent):
    agent_size = agent["size"]
    if agent_size % 2 == 0:
        agent["size"] = agent_size / 2
    else:
        agent["size"] = agent_size / 3
    return agent

def movement_generation_gene(agent):
    movements_possible = ["left", "right", "top", "bottom"]
    movement_history = agent.get("movements", [])
    if not movement_history:
        movement_choice = random.choice(movements_possible)
    else:  
        movement_history = agent["movements"]
        count = Counter(movement_history)
        weights = {"left": 0, "right": 0, "top": 0, "bottom": 0}
        for movement in movements_possible:
            movement_count = count[movement]
            if movement_count % 2 == 0:
                weights[movement] = weights[movement] + 0.35
            else:
                weights[movement] = weights[movement] + 0.30
        ordered_weights = [weights[m] for m in movements_possible]
        movement_choice = random.choices(movements_possible, weights=ordered_weights)[0]
    return movement_choice

### Agent Initialization

Creates an agent with a unique ID, size, position on the grid, and a history log for tracking development over generations.


In [4]:
# agent.py

def agent_definition(seed_id, size, position, history):
    agent = {
        "id": seed_id, 
        "size": size,
        "position": position, 
        "history": history
    }
    return agent

### Development Cycle

Simulates one development step for each agent:
- Updates size based on movement direction.
- Moves the agent within environment bounds.
- Logs new position, size, and movement in agent history.


In [5]:
# development.py

def development_run(list_of_agents, env):
    env = env
    for agent in list_of_agents: 
        current_agent_size = agent["size"]
        agent_movement_choice = movement_generation_gene(agent)
        if agent_movement_choice in ["left", "top"]: 
            current_agent_size = current_agent_size + (current_agent_size/2) 
        elif agent_movement_choice in ["bottom", "right"]:
            current_agent_size = current_agent_size + (current_agent_size/3)
        agent["size"] = current_agent_size

        # Updating agent position
        x, y = agent["position"]
        if agent_movement_choice == "left":
            x -= 1
        elif agent_movement_choice == "right":
            x += 1
        elif agent_movement_choice == "top":
            y -= 1
        elif agent_movement_choice == "bottom":
            y += 1

        x_max, y_max = env
        x = max(0, min(x, x_max - 1))
        y = max(0, min(y, y_max - 1))
        
        agent["position"] = (x, y)

        # Updating agent history
        agent["history"].append({
        "position": agent["position"],
        "size": agent["size"],
        "move": agent_movement_choice
        })
        
        # Track movement choice in a dedicated list
        if "movements" not in agent:
            agent["movements"] = []
        agent["movements"].append(agent_movement_choice)
        
    return list_of_agents

### Generation Runner

Runs a single generation by calling the development cycle for all agents. Returns updated agent states after development.


In [6]:
# generations.py

def generation_run(list_of_agents, env):
    env = env
    agents_after_development = development_run(list_of_agents=list_of_agents, env=env)
    # Need to write code here for updating position and history and displaying movement
    return agents_after_development

### Main Simulation Loop

Sets up the environment and agents, then runs the simulation across multiple generations.  
Prints agent sizes, positions, and full movement history after completion.


In [7]:
# Setting up main loop
def main():

    # Step 1: Set up the environment
    env = return_environment(5, 5)  # 5x5 grid

    # Step 2: Instantiate agents
    agent_1 = agent_definition(
        seed_id="agent_1",
        size=3,
        position=(1, 2),
        history=[]
    )

    agent_2 = agent_definition(
        seed_id="agent_2",
        size=3,
        position=(2, 1),
        history=[]
    )

    list_of_agents = [agent_1, agent_2]

    # Step 3: Define number of generations
    num_generations = 15

    # Step 4: Run simulation
    for gen in range(num_generations):
        print(f"\n--- Generation {gen + 1} ---")

        # Run development cycle for this generation
        list_of_agents = development_run(list_of_agents, env)

        # Visualize the grid
        print_grid(list_of_agents, env)

        # Print agent status after this generation
        # for agent in list_of_agents:
            # last_move = agent["movements"][-1] if "movements" in agent and agent["movements"] else "N/A"
            # print(f"{agent['id']} - Position: {agent['position']} | Size: {agent['size']:.2f} | Move: {last_move}")
        
    print("\n--- Full Movement History ---")
    for agent in list_of_agents:
        print(f"\n{agent['id']} movement history:")
        print(" → ".join(agent["movements"]))
        
if __name__ == "__main__":
    main()


--- Generation 1 ---

Agent Sizes:
agent_1: 🟩🟩 (4.50) (['left'])
agent_2: 🟩🟩 (4.00) (['bottom'])

--- Generation 2 ---

Agent Sizes:
agent_1: 🟩🟩🟩 (6.00) (['left', 'bottom'])
agent_2: 🟩🟩 (5.33) (['bottom', 'right'])

--- Generation 3 ---

Agent Sizes:
agent_1: 🟩🟩🟩🟩 (8.00) (['left', 'bottom', 'bottom'])
agent_2: 🟩🟩🟩🟩 (8.00) (['bottom', 'right', 'left'])

--- Generation 4 ---

Agent Sizes:
agent_1: 🟩🟩🟩🟩🟩🟩 (12.00) (['left', 'bottom', 'bottom', 'top'])
agent_2: 🟩🟩🟩🟩🟩🟩 (12.00) (['bottom', 'right', 'left', 'top'])

--- Generation 5 ---

Agent Sizes:
agent_1: 🟩🟩🟩🟩🟩🟩🟩🟩🟩 (18.00) (['left', 'bottom', 'bottom', 'top', 'top'])
agent_2: 🟩🟩🟩🟩🟩🟩🟩🟩🟩 (18.00) (['bottom', 'right', 'left', 'top', 'top'])

--- Generation 6 ---

Agent Sizes:
agent_1: 🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩 (27.00) (['left', 'bottom', 'bottom', 'top', 'top', 'top'])
agent_2: 🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩 (27.00) (['bottom', 'right', 'left', 'top', 'top', 'top'])

--- Generation 7 ---

Agent Sizes:
agent_1: 🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩 (36.00) (['left', 'bottom', 'bottom', 'top